In [1]:
import numpy as np
import pandas as pd
import numpy.random as rgt
import matplotlib.pyplot as plt
import scipy.stats as spstat
import time
import math

from sqr_scqr_conquer import low_dim, high_dim
from cqr_admm import cqrpadmm
from ranklasso import ranklassoregression

def cov_generate(std, corr=0.5):
    p = len(std)
    R = np.zeros(shape=[p,p])
    for j in range(p-1):
        R[j, j+1:] = np.array(range(1, len(R[j,j+1:])+1))
    R += R.T
    return np.outer(std, std)*(corr*np.ones(shape=[p,p]))**R

rgt.seed(835)

In [2]:
n, p = 90,350
Mu, Sig = np.zeros(p), cov_generate(np.ones(p))
#Mu,Sig= np.zeros(p),0.5*np.ones(shape=(p,p))+0.5*np.eye(p)
beta = np.zeros(p)
beta[[0, 1,4]] =[3,1.5,2]
#beta[[0, 1,2]] =[math.sqrt(3),math.sqrt(3),math.sqrt(3)]
tau=np.zeros(19)
niter=20000
for i in range(0,19):
    tau[i]=0.05*(i+1)
M = 10
coef_err = np.zeros(shape=(M,4))
true_pos = np.zeros(shape=(M,4))
false_pos = np.zeros(shape=(M,4))
true_beta = np.copy(beta)
true_set = np.where(true_beta!=0)[0]
runtime = np.zeros(4)
for m in range(M):
    X = rgt.multivariate_normal(mean=Mu, cov=Sig, size=n)
    
    #Y = X.dot(beta)+rgt.normal(0,math.sqrt(3), size=n)
    #Y = X.dot(beta)+math.sqrt(6)*(0.5*rgt.normal(0,1,size=n)+0.5*rgt.normal(0,0.5**3,size=n))
    Y = X.dot(beta)+rgt.standard_t(3,size=n)
    #Y = X.dot(beta)+rgt.standard_cauchy(size=n)
    Xcenter=X-X.mean(axis=0)
    
    hdscqr = high_dim(Xcenter, Y, intercept=False)
    hdcqr = cqrpadmm(Xcenter, Y, intercept=False)
    hdranklasso = ranklassoregression(Xcenter, Y, intercept=False)
    ## Rank Lasso 
    tic = time.time()
    ranklasso = hdranklasso.ranklasso(method='highs')
    runtime[0] += time.time() - tic
    ranklasso_size, ranklasso_set = sum(ranklasso['beta'] != 0), np.where(ranklasso['beta'] != 0)[0]        
    coef_err[m,0] = (ranklasso['beta'] - true_beta).dot(Sig).dot(ranklasso['beta'] - true_beta)    
    true_pos[m,0] = len(np.intersect1d(true_set, ranklasso_set))    
    false_pos[m,0] = ranklasso_size - true_pos[m,0]   
    
    ## l1-penalized cqr_admm 
    tic = time.time()
    cqradmm = hdcqr.cqrp_admm_smw(tau=tau, maxit=niter)
    runtime[1] += time.time() - tic
    admm_size, admm_set = sum(cqradmm['beta'] != 0), np.where(cqradmm['beta'] != 0)[0]        
    coef_err[m,1] = (cqradmm['beta'] - true_beta).dot(Sig).dot(cqradmm['beta'] - true_beta)    
    true_pos[m,1] = len(np.intersect1d(true_set, admm_set))    
    false_pos[m,1] = admm_size - true_pos[m,1]   
    
    ## l1-penalized scqr 
    tic = time.time()
    l1_model = hdscqr.cqr_l1(tau=tau,standardize=True)
    runtime[2] += time.time() - tic
    l1_size, l1_set = sum(l1_model['beta'] != 0), np.where(l1_model['beta'] != 0)[0]
    coef_err[m,2] = (l1_model['beta'] - true_beta).dot(Sig).dot(l1_model['beta'] - true_beta)    
    true_pos[m,2] = len(np.intersect1d(true_set, l1_set))    
    false_pos[m,2] = l1_size - true_pos[m,2]   
    
    ## irw scqr
    tic = time.time()
    irw_model = hdscqr.cqr_irw(tau=tau,standardize=True)
    runtime[3] += time.time() - tic
    irw_size, irw_set = sum(irw_model['beta'] != 0), np.where(irw_model['beta'] != 0)[0]
    coef_err[m,3] = (irw_model['beta'] - true_beta).dot(Sig).dot(irw_model['beta'] - true_beta)    
    true_pos[m,3] = len(np.intersect1d(true_set, irw_set))    
    false_pos[m,3] = irw_size - true_pos[m,3]   

    
out1 = np.array([np.mean(coef_err, axis=0), np.std(coef_err, axis=0), np.mean(true_pos, axis=0), np.mean(false_pos, axis=0), runtime/M])

out1 = pd.DataFrame(out1, columns=['ranklasso','cqr_admm','scqr-l1','scqr-scad'],
                    index=['model err', '(std)', 'true pos', 'false pos', 'runtime'])


In [3]:
out1

,ranklasso,cqr_admm,scqr-l1,scqr-scad
model err,0.982420,0.480869,0.365936,0.064203
(std),0.504313,0.155009,0.126416,0.043827
true pos,3.000000,3.000000,3.000000,3.000000
false pos,0.600000,5.300000,10.000000,0.000000
runtime,80.843895,7.920292,0.459379,0.382571
